In [ ]:
import numpy as np
import numpy.random as nrand
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import copy
import pickle
import multiprocessing


In [ ]:
# *** GENERATING INTERACTION MATRICES ***************************************

def imatrix_rand(N,K):
    '''
    This function takes the number of N elements and K interdependencies
    and creates a random interaction matrix.
    '''
    Int_matrix_rand = np.zeros((N, N))
    for aa1 in np.arange(N):
        Indexes_1 = list(range(N))
        Indexes_1.remove(aa1)  # remove self
        np.random.shuffle(Indexes_1)
        Indexes_1.append(aa1)
        Chosen_ones = Indexes_1[-(K+1):]  # this takes the last K+1 indexes
        for aa2 in Chosen_ones:
            Int_matrix_rand[aa1, aa2] = 1  # we turn on the interactions with K other variables
    return(Int_matrix_rand)

def calc_fit(NK_land_, inter_m, Current_position, Power_key_):
    '''
    Takes the landscape and a given combination and returns a vector of fitness
    values for the vector of the N decision variables.
    '''
    Fit_vector = np.zeros(N)
    for ad1 in np.arange(N):
        Fit_vector[ad1] = NK_land_[np.sum(Current_position * inter_m[ad1]
                                          * Power_key_), ad1]
    return(Fit_vector)

def comb_and_values(NK_land_, Power_key_, inter_m):
    '''
    Calculates values for all combinations on the landscape. The resulting
    array contains:
    - the first columns indexed from 0 to N-1 are for each of the combinations
    - the column indexed N is for the total fit (average of the entire vector)
    '''
    Comb_and_value = np.zeros((2**N, N+1))  # to capture the results
    c1 = 0  # starting counter for location
    for c2 in itertools.product(range(2), repeat=N):
        # this takes time so be carefull with landscapes of bigger size
        Combination1 = np.array(c2)  # taking each combination
        fit_1 = calc_fit(NK_land_, inter_m, Combination1, Power_key_)
        Comb_and_value[c1, :N] = Combination1  # combination and values
        Comb_and_value[c1, N] = np.mean(fit_1)
        c1 = c1 + 1
    return(Comb_and_value)

In [ ]:
def normalize(array):
    MAX = np.max(array)
    MIN = np.min(array)
    return (array - MIN)/(MAX - MIN)

In [ ]:
# NK landscape parameters -----------------------------------------
# Change parameters to get fitness landscape of different variable site.

N = 10  # number of variable site


In [ ]:
NK_landscape_list = {i:[] for i in range(1,101)}
Power_key = np.power(2, np.arange(N - 1, -1, -1))
K_loop = itertools.cycle(range(1,N))
for i in range(1,101):
    for j in range(10):
        Landscape_data = []
        K = next(K_loop)
        Int_matrix = imatrix_rand(N,K).astype(int)
        NK_land = np.random.rand(2**N, N)
        NK_landscape_list[i].append(comb_and_values(NK_land, Power_key, Int_matrix))
        NK_landscape_list[i][j][:,N] = normalize(NK_landscape_list[i][j][:,N])
        print(f"{i}:{j}     ",end='\r')

In [10]:
with open(f'../FL_data_100X10/NK_{N}_landscape_list_100X10.pkl','wb') as f:
    pickle.dump(NK_landscape_list,f)